# 线性回归简洁实现

学习内容
- 使用pytorch中的utils.Data来实现数据集的封装，并使用DataLoader进行数据的载入
- 在pytorch中，学习使用torch.nn模型搭建(搭积木）
- 模型参数初始化，torch.nn.init
- 使用pytorch中的优化器,torch.optim
- 训练模型，体验网络搭建与训练

# 你将学到有关pytorch的

- torch.utils
    - torch.utils.data
        - Data.TensorDataset
        - Data.DataLoader(dataset,batch_size,shuffle=True)
- torch.nn
    - torch.nn.Linear()
        - constant_()
        - weight_()
    - torch.nn.init
    - torch.nn.MSEloss()
    - torch.nn.Sequential()
- torch.optim
    - param_group['lr']*=0.1(in optimizer.param_groups)调整学习率

In [7]:
import torch
import numpy as np
import torch.nn as nn

In [3]:
examples_num = 1000
input_num = 2
x = torch.tensor(np.random.normal(0,1,(examples_num,input_num)),dtype = torch.float32)#1000*2
true_w = torch.tensor([3.4,-7.2],dtype=torch.float32).view(2,1) #2*1
true_b = torch.tensor(3.8,dtype = torch.float32)
labels = torch.mm(x,true_w)+true_b
print(x.size())
print(labels.size())

torch.Size([1000, 2])
torch.Size([1000, 1])


# 读取数据

In [4]:
import torch.utils.data as Data

In [5]:
batch_size = 64
dataset = Data.TensorDataset(x,labels)
data_iter = Data.DataLoader(dataset,batch_size,shuffle=True)

In [6]:
for i,j in data_iter:
    print(i.size())
    print(j.size())
    print("--------")

torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([64, 2])
torch.Size([64, 1])
--------
torch.Size([40, 2])
torch.Size([40, 1])
--------


# 定义模型

In [20]:
#定义方法一 先声明层然后在forward里面进行封装
class LinearNet(nn.Module):
    #继承类，定义层
    def __init__(self,n_features):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(n_features,1)
    
    #前向传播
    def forward(self,x):
        y = self.linear(x)
        return y

In [21]:
net = LinearNet(10)
print(net)    #线性层模型

LinearNet(
  (linear): Linear(in_features=10, out_features=1, bias=True)
)


In [22]:
#写法二 直接封装
net = nn.Sequential(nn.Linear(input_num,1))
print(net)

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)


In [23]:
#写法三:先创建后加
net = nn.Sequential()
net.add_module('linear',nn.Linear(input_num,1))
print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


net.parameters()可查看可学习参数

In [24]:
#打印模型参数看一看
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.4857, -0.4204]], requires_grad=True)
Parameter containing:
tensor([-0.0931], requires_grad=True)


# 初始化模型参数

In [25]:
#初始化
from torch.nn import init
init.normal_(net[0].weight,mean=0,std=0.01)   #权重初始化
init.constant_(net[0].bias,val=0)   #b的初始化，constant常量等效于net[0].bias.data.fill_(0)
#注：net[0]表示网络第一层，xxxx_()这样的函数一般表示对前一个属性执行后一个操作

Parameter containing:
tensor([0.], requires_grad=True)

# 定义损失函数

In [31]:
loss = nn.MSELoss()
#均方差损失

# 定义优化算法

In [27]:
import torch.optim as optim
#定义随机梯度下降的优化算法
optimizer = optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [28]:
#对参数进行相应调整应该怎么办？
for param_group in optimizer.param_groups:
    param_group['lr']*=0.1
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.003
    momentum: 0
    nesterov: False
    weight_decay: 0
)


# 训练模型

In [30]:
import tqdm
from tqdm import trange

In [32]:
epochs = 1000
for epoch in trange(epochs):
    for train_data,train_labels in data_iter:
        out = net(train_data)
        l = loss(out,train_labels)
        optimizer.zero_grad() #optimizer中参数梯度清零
        l.backward()   #反向求导
        optimizer.step() #参数迭代更新
    print("epoach:",epoch,"loss",l.item())

  0%|                                                                                                                                      | 0/1000 [00:00<?, ?it/s]F:\Anaconda3\envs\pytorch\lib\site-packages\torch\autograd\__init__.py:149: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:115.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
  1%|█▍                                                                                                                           | 11/1000 [00:00<00:39, 25.06it/s]

epoach: 0 loss 69.87376403808594
epoach: 1 loss 51.8515739440918
epoach: 2 loss 30.370433807373047
epoach: 3 loss 27.279865264892578
epoach: 4 loss 37.68402862548828
epoach: 5 loss 22.00126075744629
epoach: 6 loss 24.985698699951172
epoach: 7 loss 12.386274337768555
epoach: 8 loss 17.4079647064209
epoach: 9 loss 13.625602722167969
epoach: 10 loss 10.316607475280762
epoach: 11 loss 10.334577560424805
epoach: 12 loss 7.4951276779174805
epoach: 13 loss 4.4890875816345215
epoach: 14 loss 5.054732322692871
epoach: 15 loss 2.402777910232544
epoach: 16 loss 3.9902853965759277
epoach: 17 loss 3.1321213245391846
epoach: 18 loss 1.6493886709213257
epoach: 19 loss 1.9161888360977173
epoach: 20 loss 0.9567724466323853


  3%|████▎                                                                                                                        | 34/1000 [00:00<00:15, 62.86it/s]

epoach: 21 loss 1.7708686590194702
epoach: 22 loss 0.8830572962760925
epoach: 23 loss 0.7687305808067322
epoach: 24 loss 0.7929588556289673
epoach: 25 loss 0.4623769223690033
epoach: 26 loss 0.4898095726966858
epoach: 27 loss 0.4428015649318695
epoach: 28 loss 0.36360159516334534
epoach: 29 loss 0.2333991974592209
epoach: 30 loss 0.24773840606212616
epoach: 31 loss 0.1940213441848755
epoach: 32 loss 0.16140256822109222
epoach: 33 loss 0.16314662992954254
epoach: 34 loss 0.07612945139408112
epoach: 35 loss 0.10286961495876312
epoach: 36 loss 0.0854451060295105
epoach: 37 loss 0.06279196590185165
epoach: 38 loss 0.04335251823067665
epoach: 39 loss 0.04996829479932785
epoach: 40 loss 0.049746301025152206
epoach: 41 loss 0.044433292001485825


  6%|███████                                                                                                                      | 56/1000 [00:01<00:11, 80.62it/s]

...

epoach: 947 loss 6.747640779458663e-10
epoach: 948 loss 6.422993803489874e-10
epoach: 949 loss 9.58021884045479e-10
epoach: 950 loss 4.175845502718545e-10
epoach: 951 loss 6.066997459974743e-10
epoach: 952 loss 6.331333790576821e-10
epoach: 953 loss 7.22066295644197e-10
epoach: 954 loss 5.820354198604605e-10
epoach: 955 loss 6.787416739761909e-10
epoach: 956 loss 5.940350433775166e-10
epoach: 957 loss 7.096460086231104e-10
epoach: 958 loss 6.509537908705454e-10
epoach: 959 loss 4.99730368197504e-10
epoach: 960 loss 6.112756412157694e-10
epoach: 961 loss 5.431999849925262e-10
epoach: 962 loss 7.601997364936608e-10
epoach: 963 loss 4.050093593832571e-10
epoach: 964 loss 6.304120003797209e-10
epoach: 965 loss 5.271800773698487e-10
epoach: 966 loss 7.223789344479314e-10
epoach: 967 loss 5.597470820184469e-10
epoach: 968 loss 4.481918725041112e-10


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎ | 986/1000 [00:09<00:00, 105.65it/s]

epoach: 969 loss 5.851276685397977e-10
epoach: 970 loss 7.73320629754437e-10
epoach: 971 loss 7.415493774587389e-10
epoach: 972 loss 6.144148523290482e-10
epoach: 973 loss 7.607798835351787e-10
epoach: 974 loss 6.745864422619263e-10
epoach: 975 loss 4.777561679603082e-10
epoach: 976 loss 7.057465722937195e-10
epoach: 977 loss 5.08137532051478e-10
epoach: 978 loss 5.744897890735956e-10
epoach: 979 loss 5.773699851552294e-10
epoach: 980 loss 5.339249042890515e-10
epoach: 981 loss 6.274575303777397e-10
epoach: 982 loss 5.824191129377709e-10
epoach: 983 loss 7.237076493638028e-10
epoach: 984 loss 7.370289378805239e-10
epoach: 985 loss 6.655227480223402e-10
epoach: 986 loss 5.965578031563723e-10
epoach: 987 loss 4.328370439843354e-10
epoach: 988 loss 6.732960855515557e-10


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 101.50it/s]

epoach: 989 loss 7.420396519464134e-10
epoach: 990 loss 5.428262284112861e-10
epoach: 991 loss 3.9534170381827494e-10
epoach: 992 loss 5.706340400202237e-10
epoach: 993 loss 6.298890298239712e-10
epoach: 994 loss 6.168548449814182e-10
epoach: 995 loss 6.692403298202976e-10
epoach: 996 loss 7.197712426076919e-10
epoach: 997 loss 6.554629061739092e-10
epoach: 998 loss 7.830650017304208e-10
epoach: 999 loss 6.912103112099999e-10


In [37]:
for param in net[0].parameters():
    print(param)

Parameter containing:
tensor([[ 3.4000, -7.2000]], requires_grad=True)
Parameter containing:
tensor([3.8000], requires_grad=True)
